# INSTALL JULIA - COLAB

PEGUE O SCRIPT DE TEMPLATE [AQUI](https://colab.research.google.com/drive/1lLHnAUl1kqPB6Fc1tF9r19om24DbYYFh)!

In [ ]:
install_julia = false
check_julia = false

In [ ]:
# function julia_installer()
#     %%shell
#     set -e

#     #---------------------------------------------------#
#     JULIA_VERSION="1.10.4" # any version ≥ 0.7.0
#     JULIA_PACKAGES="IJulia BenchmarkTools Plots Statistics DataStructures"
#     JULIA_NUM_THREADS=2
#     #---------------------------------------------------#

#     if [ -z `which julia` ]; then
#     # Install Julia
#     JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
#     echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
#     BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
#     URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
#     wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
#     tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
#     rm /tmp/julia.tar.gz

#     # Install Packages
#     for PKG in `echo $JULIA_PACKAGES`; do
#         echo "Installing Julia package $PKG..."
#         julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
#     done

#     # Install kernel and rename it to "julia"
#     echo "Installing IJulia kernel..."
#     julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
#         "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
#     KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
#     KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
#     mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

#     echo ''
#     echo "Successfully installed `julia -v`!"
#     echo "Please reload this page (press Ctrl+R or F5) then"
#     echo "jump to the 'Checking the Installation' section."
#     fi
# end

In [ ]:
if install_julia
    julia_installer()
end

if check_julia
    versioninfo()
end

# PROJETO 1 - ANALISE DE ALGORITMO

In [ ]:
using Plots
using BenchmarkTools

## SEARCH FUNTIONS

In [ ]:
# Simple linear search function
function simple_search(vector::Vector{Int64}, key::Int64)
    for i in 1:length(vector)
        if key == vector[i] 
            return i  # Returns the index where the key was found
        end
    end
    return -1  # Returns -1 if the key is not found
end

In [ ]:
# Optimized linear search function
function optimized_search(vector::Vector{Int64}, key::Int64)
    for i in 1:length(vector)
        if key == vector[i]
            return i  # Returns the index where the key was found
        elseif key < vector[i]
            return -1  # Returns -1 if the key is not found
        end
    end
    return -1  # Returns -1 if the key is not found
end

In [ ]:
# Binary search function
function binary_search(vector::Vector{Int64}, key::Int64)
    low, high = 1, length(vector)

    while low <= high
        mid = div(low + high, 2)
        if vector[mid] == key
            return mid  # Returns the index where the key was found
        elseif vector[mid] < key
            low = mid + 1
        else
            high = mid - 1
        end
    end
    return -1  # Returns -1 if the key is not found
end

## VECS & LISTS

In [ ]:
# VARS
n_values = [10^i for i in 4:7]  # Vectors Sizes
q_values = [10^i for i in 2:5]  # Amount of Keys

# Initialize the list of vectors and keys
vector_list = [rand(1:10^5, n) for n in n_values]
key_list = [collect(1:1:q) for q in q_values] # K2[1-100], ... , K5[1-10^5]

# Initialize the list of timers
time_simple_search = zeros(length(n_values), length(q_values))
time_optimized_search = zeros(length(n_values), length(q_values))
time_binary_search = zeros(length(n_values), length(q_values))
time_sort = zeros(length(n_values))

## BENCHMARK

In [ ]:
# Simple linear search - Benchmarking
for i in 1:length(vector_list)  # For each vector size
    for j in 1:length(key_list)  # For each key numbers
        bench_simple = @benchmark [simple_search($vector_list[$i], k) for k in $key_list[$j]]
        time_simple_search[i, j] = median(bench_simple).time / 1e9
    end
end

In [ ]:
# Sorting the vectors - Benchmarking
for i in 1:length(vector_list)   # For each vector size
    bench_sort = @benchmark sort($vector_list[$i]) # Benchmark sorting
    time_sort[i] = median(bench_sort).time / 1e9 # Elepsed Time
    vector_list[i] = sort(vector_list[i]) # Sort the vector
end

In [ ]:
# Optimized Linear search - Benchmarking
for i in 1:length(vector_list)   # For each vector size
    for j in 1:length(key_list)  # For each key numbers
        bench_optimized = @benchmark [optimized_search($vector_list[$i], k) for k in $key_list[$j]]
        time_optimized_search[i, j] = median(bench_optimized).time / 1e9
    end
end

In [ ]:
# Binary search - Benchmarking
for i in 1:length(vector_list)   # For each vector size
    for j in 1:length(key_list)  # For each key numbers
        bench_binary = @benchmark [binary_search($vector_list[$i], k) for k in $key_list[$j]]
        time_binary_search[i, j] = median(bench_binary).time / 1e9
    end
end

## PLOTS

In [ ]:
xtick_positions = [0, 25000, 50000, 75000, 10^5]
xtick_labels = ["0", "25.000", "50.000", "75.000", "100.000"]

# Ploting graphics
for i in 1:4
    p = plot(
        q_values,
        time_simple_search[i, :],
        label = "Simples",
        xlabel = "Numero de Busca (q)",
        ylabel = "Tempo (ns)",
        title = "Tamanho do Vetor (n) = $(n_values[i])",
        legend = :topleft,
        xticks = (xtick_positions, xtick_labels),
        yscale = :log10,  #log scale
        lw = 2
    )
    plot!(q_values, time_optimized_search[i, :], label = "Otimizada", lw = 2)
    plot!(q_values, time_binary_search[i, :], label = "Binária", lw = 2)
    plot!(q_values, time_sort, label = "Ordenarção", lw = 2)

    display(p)
    savefig(p, "../imgs/log_10^$(i+3).png")
end

In [ ]:
# for i in 1:4
#     # Gráfico com escala linear
#     p = plot(
#         q_values,
#         time_binary_search[i, :],
#         label = "Binária",
#         xlabel = "Número de Busca (q)",
#         ylabel = "Tempo (ns)",
#         title = "Tamanho do Vetor (n) = $(n_values[i])",
#         legend = :topleft,
#         xticks = (xtick_positions, xtick_labels),
#         lw = 2
#     )
#     plot!(q_values, time_optimized_search[i, :], label = "Otimizada", lw = 2)
#     plot!(q_values, time_sort, label = "Ordenação", lw = 2)
#     plot!(q_values, time_simple_search[i, :], label = "Simples", lw = 2)

#     display(p)
#     savefig(p, "../imgs/linear_10^$(i+3).png")
# end